# MAKE.. A Machine That Can Write

This notebook is heavily based on the notebook 'Train a GPT-2 Text-Generating Model w/ GPU For Free' by [Max Woolf](http://minimaxir.com), version last updated by Max on 14 February 2021.

This fork was made by [James Baker](https://cradledincaricature.com/) for the session "[MAKE.. A Machine That Can Write](https://www.eventbrite.co.uk/e/make-a-machine-that-can-write-tickets-187702823197?aff=ebdsoporgprofile)", first held at the Digital Humanities Hub, University of Southampton, on 22 February 2022. The latest revision was for a October 2022 workshop.

This notebook enables you to retrain an advanced text generating neural network on any text dataset **for free on a GPU using Collaboratory** using `gpt-2-simple`. To get the most out of it, you need a Google Account (and you don't have and don't want a Google Account, pair up with some who has).

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read Max's [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!

To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Optional: using the default options below the model will be trained on a corpus of texts written by William Shakespeare. Alternatively, choose a different corpus of texts (ideally in .txt format) in whose style/voice you want the machine to write.
  - James has previously used a corpus of catalogue descriptions (circa 1.5m words) written by the historian Dorothy George between 1930 and 1954: data https://zenodo.org/record/3245017; results https://cradledincaricature.com/2020/06/18/mary-dorothy-george/
  - To create your own, good literary examples can be found at [Project Gutenberg](https://www.gutenberg.org/).
4. Run the cells below: that is, hit the play buttons in order, waiting for each to complete (when a tick appears to the left of the code block) before moving onto the next):


In [ ]:
%tensorflow_version 2.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


## GPU

Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [ ]:
!nvidia-smi

Tue Oct 18 13:21:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80/P4 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [ ]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 314Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:01, 966kit/s]
Fetching hparams.json: 1.05Mit [00:00, 541Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:47, 10.4Mit/s]
Fetching model.ckpt.index: 1.05Mit [00:00, 332Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 1.41Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 1.20Mit/s]


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it mAY ask for an auth code; that auth is not saved anywhere)

After entering you auth code, hit enter.

In [ ]:
gpt2.mount_gdrive()

Mounted at /content/drive


## Uploading a Text File to be Trained to Colaboratory

First, download a dataset with which to inflect the langauge model. This notebook is setup to run against '[obama_sotu.txt](https://drive.google.com/file/d/184DsOBtHrQxAQTQiN7ZRVrk6tXvPpWeG/view?usp=sharing)', a dataset of Obama's State of the Union addresses. Alternatively, you could use:


*   '[bjork-radiohead-prince.txt](https://drive.google.com/file/d/1JVJBaRPX7j1lqARzprXMQQX203RRwyYB/view?usp=sharing)', a small dataset of music lyrics from three artists.
*   '[shakespeare.txt](https://drive.google.com/file/d/1Id-1wra6vp2SpNg55Ak9F389jE-cH-24/view)', which is text file containing texts written by William Shakespeare.
*   Or '[MDG.txt](https://drive.google.com/file/d/15-PrlOfwOZX_ZP9WQw1Dkajrxe3OqAoq/view?usp=sharing)', a dataset of descriptions of satirical prints.

Which ever you choose it will work, you may just wish to change the variable 'simObama' to something different for ease of reading (that is, when you see the variable 'simObama' that could be anything, it just makes sense to make it something that relates to your data).

Then in the Colaboratory Notebook sidebar on the left of the screen, select *Files* (the folder icon), hit the upload icon, and upload your chose to your notebook. Once the file appears in the sidebar you are ready to go.

**Note**: if you upload a file other than 'obama_sotu.txt', be sure to change the file name in the cell below before you run the cell.

In [ ]:
file_name = "obama_sotu.txt"

**Ignore the next three code blocks if you've uploaded your dataset using `file_name = "obama_sotu.txt"` above**

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
file_name = "/content/drive/MyDrive/Colab_Notebooks/obama_sotu.txt"

In [ ]:
print(file_name)

obama_sotu.txt


## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

For 'obama_sotu.txt' (which is roughly 300kb, 50k words) you should start seeing snippets of text after 2 minutes. You should notice that each printed snippet is "better" than the last (though we will talk about what "better" means in this context). The model will complete after around 5 minutes for 'obama_sotu.txt' (more like 30 minutes for 'shakespeare.txt'.

If you've uploaded a larger dataset to train the model, note that the training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [ ]:
import tensorflow as tf
tf.compat.v1.reset_default_graph()

sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=100,
              restore_from='fresh',
              run_name='simObama',
              print_every=10,
              sample_every=20,
              save_every=100
              )

Loading checkpoint models/124M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  2.33it/s]


dataset has 65600 tokens
Training...
[10 | 26.07] loss=2.53 avg=2.53
[20 | 46.80] loss=2.35 avg=2.44
======== SAMPLE 1 ========
 son and daughter in a country where we share the values we hold proud.

We've also begun to move forward in a number of ways. Last week I sent my administration the first comprehensive immigration reform bill in years. Because our immigration system reflects our ideals, we will make sure our children are sent to college and meet the high expectations of their nation's top university.

And we've added more than 1 million families and made millions every year. It does not require our politicians to act. No more job-killing regulations. No more tax cuts. No more spending cuts. No more tax cuts for the wealthiest 5 percent of Americans. In fact, over the last 3 years, we've actually saved the American people more money by reducing our deficit by 13 percent.

And we've set a higher expectation for the future of our children. For over 2 decades, every child in this

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it to Google Drive first, then download from there. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='simObama')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

**If you've just completed creating you model, skip to Generate Text From The Trained Model**

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name='simShakes')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='simShakes')

Loading checkpoint checkpoint/simShakes/model-500
INFO:tensorflow:Restoring parameters from checkpoint/simShakes/model-500


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [ ]:
gpt2.generate(sess, run_name='simObama')

A new study from the University of Michigan shows that in countries where gay marriage is legal, HIV/AIDS deaths decline, they don't. And that's because they have to.

The study, by researchers from the U.S. Centers for Disease Control and Prevention, found that between 2005 and 2011, the average family in the United States experienced a 50 percent decline in life expectancies. That's a fact. But as soon as same-sex couples get married, we have an obligation to protect them. And so Congress should issue laws that recognize that same-sex couples can't expect to get married because they don't get married. [Laughter]

Because gay marriage is a civil rights issue, we should have the legal status of a state crime in the United States of America. But we don't. And that's why, since the very day I signed the civil rights act, I have asked this Congress to send me another law that protects our most vulnerable people from the ravages of discrimination.

Because of the laws in this book, I have 

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the creative the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [ ]:
gpt2.generate(sess,
              run_name='simObama',
              length=500,
              temperature=0.95,
              nsamples=5,
              batch_size=5,
              include_prefix=True,
              prefix="The American Dream"
              )

The American Dream is not on the horizon, and the agenda we have to follow remains the same. It's just a matter of time. 

I know that we may see profound change, even a century of decline, if we look to past institutions and policies. As we speak, our planet is warming. More so than 50 years ago, it was the warmest on record. That planet had previously been thought to be in a state of near-term starvation, but this summer, scientists have shown us that the Central Valley is actually melting. The cold that we've been driving up in the last 2 and a half years has pushed down the temperature of the air we're living in to more than 18°F (4.7°C) below pre-industrial levels. 

Those of us here tonight who care about the well-being of our planet have had enough of this "hiatus" moment. We should do what it takes to combat global warming, to protect the environment, build more resilient communities, and promote a renewed effort to develop new technologies to cool our planet. 

Looking ahead, 

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      run_name='simObama',
                      destination_path=gen_file,
                      length=300,
                      temperature=0.77,
                      prefix="My fellow Americans,",
                      nsamples=100,
                      batch_size=5
                      )

In [ ]:
# may have to run twice to get file to download
files.download(gen_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Where to go next

## Samples

Go [here](https://drive.google.com/drive/folders/1ZCCYvbG126iM1vgyh93tg74raB14YYh4?usp=sharing) for sample of texts generated by this model using 'shakespeare.txt' (temperature=0.8, prefix="LADY").

## Reading

As Ted Underwood writes in a fascinating piece ('Analytics and Imagination' Journal of Cultural Analytics blog, [15 October 2021](https://culturalanalytics.org/post/1205-analytics-and-imagination)) on the emergent challege of "technology companies hav[ing] started to realize that they’re in the business of modeling (and generating) culture":

> the dissolving boundary between analysis and creation is an interesting development for humanists

And so whilst machine generated texts have their problems (they are woefully inefficient learners, have tendancy to forget things, and lack narrative logic) they still give us uncanny versions of texts we know, offering opportunities for [deformative reading](https://doi.org/10.1353/nlh.1999.0010). I've done this in my work, using a 'sim' writer to [help me understand the ways the labour conditions in which the cataloguer and historian Dorothy George worked shaped and constrained her language choices](https://doi.org/10.1093/hisres/htaa026). These machine texts also give us insight into how predictive systems work, for more on which I thoroughly recommend Leah Henrickson, *Reading Computer-Generated Texts* (Cambridge: Cambridge University Press, [2021](doi.org/10.1017/9781108906463)). The key thing is that unless you '[overfit](https://en.wikipedia.org/wiki/Overfitting#Machine_learning)' the model it will write novel text, text that is like but not the same as the text(s) used to inflect it. What this means for empirical work, is a matter for discussion.

## Some Next Steps

In this notebook we've only used the a small GPT-2 model and yet we've been able to produce text (far from logical text, but text at least in the style/voice of the text we  used to inflect the model). GPT-2 was launched in 2019. Language-based neural networks have advanced enormously in recent years (such that they appear to be keeping Underwood awake at night!), and so any serious work would go beyond our toy example to use more recent architectures.

![alt text](https://i.imgur.com/qbVjgI3.png)

One interesting area of exploration is [asking RNNs questions](https://medium.com/analytics-vidhya/an-ai-that-does-your-homework-e5fa40c43d17). Imagine for a moment that we tried this with our model, and the kind of answers we would get. Nevertheless, with very large datasets and very large models, we could build different way of querying information: asking the model open questions, observing the answer, and the using that to look for the data that correlates. As Underwood suggests, this hybrid of creativity and analysis is a fascinating thread to pull at.

## Advanced: Generate Text From The Pretrained Model

If you want to generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M or 1558M models with this notebook.

In [ ]:
model_name = "774M"

gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 354Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 131Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 279Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 3.10Git [00:23, 131Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 380Mit/s]                                                
Fetching model.ckpt.meta: 2.10Mit [00:00, 226Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 199Mit/s]                                                       


In [ ]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

W0828 18:37:58.571830 139905369159552 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Loading pretrained model models/774M/model.ckpt


In [ ]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="The secret of life is",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

The secret of life is that it's really easy to make it complicated," said Bill Nye, the host of the popular science show "Bill Nye the Science Guy." "And this is one of the reasons why we all need to be smarter about science, because we can't keep up with the amazing things that are going on all the time."

While Nye is correct that "everything that's going on all the time" is making the world a better place, he misses the point. This is not
The secret of life is in the rhythm of the universe. It's not a mystery. It's not a mystery to me. It's the nature of the universe. It's the beauty of the universe. It's the way the universe works. It's the way the universe is. It's the way the universe is going to work. It's the way the universe is. It's the way the universe is. It's the way the universe is. It's the way the universe is. It's the way
The secret of life is in the universe.


-

The Red Devil

It's the end of the world as we know it, and the only thing that can save us is a band of 

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.